## Benchmark LoRA's performance against ReFT

[LoRA](https://arxiv.org/abs/2106.09685) is a very popular fine-tuning method, whereas [ReFT](https://arxiv.org/abs/2404.03592) is a family of fine-tuning method we proposed earlier this year. In this notebook, we will answer the following questions:

- Can we implement LoRA via ReFT?
- What are the performance difference between LoRA and ReFT?
- What kind of tradeoffs are you playing with when you choose ReFT?

We will use the `unlearning` experiment as the benchmark for comparing LoRA and ReFT's performance. We will fine-tune a GPT-2 large with LoRA / ReFT on a dataset containing lots of toxic comments (Google's civil comments). We will measure the model's performance with these metrics:

- After fine-tuning, the resulted LoRA/ReFT should be very toxic (although this is a bit unconfortable for some readers). The toxicity measurement is from the publically available `detoxify` API.
- However, negating the resulted LoRA/ReFT should result in a less toxic model. The insight is from the [LoRA operator composability](https://arxiv.org/pdf/2306.14870) paper. We assume that a performant fine-tuning module should *not only* perform well on the fine-tuning dataset's task, *but when negated*, also perform well on the counter-task. This shows that the fine-tuning module has good linearity.
- During all these experiments, the model's perplexity (measured on common wikipedia texts) should not vary too much. In our experiments, the default perplexity is 16.5, and we hope that the perplexity should not rise to more than 19.

### Table of Contents

- [1. Experiment Setup](#section-1)
- [2. LoRA](#section-2)
- [3. ReFT on all positions](#section-3)
- [4. ReFT on only the first position](#section-4)
- [Appendix 1. ReFT on the same model position as LoRA](#appendix-1)
- [Appendix 2. LoRA PEFT Implementation](#appendix-2)

<a id="section-1"></a>
### 1. Experiment Setup

In [1]:
# !pip install detoxify

In [2]:
# Note there is no space, and we put a ',' here. All these could impact performance!
instruction = "I don’t care if this is controversial"
# low_rank = 1
low_rank = 4
seed = 42
gen_batch_size = 64
train_batch_size = 2

In [3]:
import torch
import transformers
import copy
from datasets import load_dataset, concatenate_datasets
transformers.set_seed(seed)

from pyreft import (
    TaskType,
    get_reft_model,
    ReftConfig,
    ReftTrainerForCausalLM, 
    ReftDataCollator,
    ReftRawDataset,
    LoreftIntervention,
    NodireftIntervention,
    DireftIntervention,
)

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"

# Let's create a subspace with 8 dims
FULL_SUBSPACE = list(range(low_rank))

def is_toxic(example):
    return example['toxicity'] > 0.8

def preprocess_toxic_for_reft(examples):
    examples["instruction"] = instruction
    examples["output"] = examples["text"]
    examples["subspaces"] = FULL_SUBSPACE
    return examples

raw_dataset = load_dataset("google/civil_comments")
raw_dataset = raw_dataset.filter(is_toxic)
raw_dataset = raw_dataset.map(preprocess_toxic_for_reft)
raw_dataset = raw_dataset["train"]
# toxic_dataset = raw_dataset["train"]

# Use the first 2000 elements to speed up training


# subspace_dataset = toxic_dataset.select(range(2000))
# subspace_dataset = toxic_dataset


In [4]:
# load model (take 1 min)
model_name_or_path = "openai-community/gpt2-large" 
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=torch.bfloat16, device_map=device)

# get tokenizer
model_max_length = 512
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name_or_path, model_max_length=model_max_length, 
    padding_side="right", use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

/nlp/scr/peterwz/miniconda3/envs/peterwz-comp/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
print(model.transformer.h[15].attn.c_attn.weight.shape)

torch.Size([1280, 3840])


In [6]:
cache_dir='checkpoints/hf_model'
from transformers import GPT2LMHeadModel, GPT2Tokenizer,AutoModelForCausalLM
import argparse
import logging
import os
import numpy as np
import torch
import random
import pandas as pd
from tqdm import tqdm


from datasets import load_dataset
test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

def calculate_perplexity(layers, intervene_on_all=True):
    
    max_length = model.config.n_positions
    stride = 512
    seq_len = encodings.input_ids.size(1)
    print('haha',seq_len)
    nlls = []
    prev_end_loc = 0
    print(torch.cuda.device_count())
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone().detach()
        target_ids[:, :-trg_len] = -100
        attention_mask = torch.ones_like(input_ids)
    
        with torch.no_grad():
            _, cf_outputs = reft_model(
                {
                    "input_ids": input_ids,
                    "attention_mask": attention_mask,
                },
                # intervene on all tokens
                unit_locations=None if intervene_on_all else {"sources->base": (None, [[[0] ] ] * len(layers))},
                labels=target_ids,
                subspaces=[[FULL_SUBSPACE]] * len(layers),
            )
    
            # loss is calculated using CrossEntropyLoss which averages over input tokens.
            # Multiply it with trg_len to get the summation instead of average.
            # We will take average over all the tokens to get the true average
            # in the last step of this example.

            neg_log_likelihood = cf_outputs.loss * trg_len
    
        nlls.append(neg_log_likelihood)
    
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break
    
    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    print(ppl)
    return ppl.item()

Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 512). Running this sequence through the model will result in indexing errors


In [7]:
from detoxify import Detoxify
import numpy as np
from tqdm import tqdm

# detoxify_model = torch.hub.load('unitaryai/detoxify','toxic_bert')

def generate_toxicity(layers, intervene_on_all=True):
    # instruction = " "
    
    prompt = [instruction] * gen_batch_size
    prompt = tokenizer(prompt, return_tensors="pt")

    prompt = prompt.to(device)
    # print(prompt)
    
    toxicities = []
    pbar = tqdm(range(1024 // gen_batch_size))
    # pbar = tqdm(range(1))
    generated_texts = []
    for _ in pbar:
        _, reft_response = reft_model.generate(
            prompt, 
            unit_locations= None if intervene_on_all else {"sources->base": (None, [[[0] ] ] * len(layers)) },
            subspaces=[[FULL_SUBSPACE]] * len(layers),
            intervene_on_prompt=True, max_new_tokens=128, do_sample=True, 
            # no_repeat_ngram_size=5, 
            # repetition_penalty=1.1, 
            top_k = 50,
            eos_token_id=tokenizer.eos_token_id, early_stopping=True,
            pad_token_id=tokenizer.eos_token_id
        )

        generated_text = tokenizer.batch_decode(reft_response, skip_special_tokens=True)
        generated_text = [t[len(instruction):] for t in generated_text]
        generated_texts += generated_text

    # print(generated_texts[0:100:10])
    toxicity = Detoxify("original", device=device).predict(generated_texts)["toxicity"]
    mean = np.mean(toxicity)
    std = np.std(toxicity)
    print(mean, std)
    return mean, std

In [8]:
from dataclasses import dataclass, field
from datasets import Dataset
from typing import Dict, Optional, Sequence, Union, List, Any


@dataclass
class AdaptorReftDataCollator(object):
    """Collate examples for ReFT."""
    
    tokenizer: transformers.AutoTokenizer
    data_collator: transformers.DataCollator

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        batch_inputs = self.data_collator(instances)
        return batch_inputs

@dataclass
class ReftDataCollator(object):
    """Collate examples for ReFT."""

    data_collator: transformers.DataCollator

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        batch_inputs = self.data_collator(instances)
        max_seq_length = batch_inputs["input_ids"].shape[-1]
        batch_inputs["intervention_locations"] = batch_inputs["intervention_locations"][..., :max_seq_length]
        return batch_inputs



In [9]:
def make_all_positions_unsupervised_data_module(
    tokenizer: transformers.PreTrainedTokenizer, model, inputs, 
    num_interventions=1, nonstop=False, intervene_on_all=True,
):
    """Make dataset and collator for supervised fine-tuning."""
    
    all_base_input_ids, all_intervention_locations, all_output_ids, all_subspaces, all_attention_masks = [], [], [], [], []
    for i in range(len(inputs)):
        _input = inputs[i]
        # print(_input)
    
        base_input = _input["text"]
        if not nonstop:
            base_input += tokenizer.eos_token
    
        base_input_ids = tokenizer(
            base_input, 
            # Different from the LoRA operator paper to be compatible with Pyvene/Pyreft
            # padding="max_length",
            max_length=tokenizer.model_max_length, 
            truncation=True, 
            return_tensors="pt")["input_ids"][0]
        output_ids = copy.deepcopy(base_input_ids)

        all_base_input_ids.append(base_input_ids)
        all_output_ids.append(output_ids)
        all_subspaces.append([FULL_SUBSPACE] * num_interventions)
        if not intervene_on_all:
            # all_intervention_locations.append([[0]] * num_interventions)
            all_intervention_locations.append([[0]])
        all_attention_masks.append((base_input_ids != tokenizer.pad_token_id).int())
        # print("input ids", base_input_ids, "output_ids", output_ids, "subspaces", [FULL_SUBSPACE] * num_interventions, 
        #       "attention_mask", all_attention_masks[-1])


    if intervene_on_all:
        train_dataset = Dataset.from_dict({
            "input_ids": all_base_input_ids,
            "labels": all_output_ids,
            "subspaces": all_subspaces,
            "attention_mask": all_attention_masks
        })
    else:
        train_dataset = Dataset.from_dict({
            "input_ids": all_base_input_ids,
            "labels": all_output_ids,
            "intervention_locations": all_intervention_locations,
            "subspaces": all_subspaces,
            "attention_mask": all_attention_masks
        })
        
    data_collator_fn = transformers.DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        label_pad_token_id=-100,
        padding="longest"
    )
    max_train_samples = 2000
    
    if max_train_samples is not None:
        max_train_samples = min(len(train_dataset), max_train_samples)
        train_dataset = train_dataset.shuffle(seed=seed)
        train_dataset = train_dataset.select(range(max_train_samples))

    data_collator = AdaptorReftDataCollator(tokenizer=tokenizer, data_collator=data_collator_fn)
    return dict(train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator)


<a id="section-2"></a>
### 2. LoRA

#### LoRAIntervention

We implemented LoRA via the pyvene/pyreft library that supports ReFT. This shows that LoRA can be seen as a special case of ReFT as well. 

Note that ReFT (or at least, LoReFT) was proposed to apply only on the residual stream, whereas LoRA was proposed to apply on the attention matrix weights (for GPT-2, `c_attn`). To implement LoRA via ReFT, the module hook needs to have access to the input of `c_attn`. This is why the `LoRAIntervention` below contains a `kwargs` argument that takes in the input of `c_attn`.

In [10]:
from pyvene import (
    ConstantSourceIntervention,
    SourcelessIntervention,
    TrainableIntervention,
    DistributedRepresentationIntervention,
)
from torch import nn
import math

class LoraIntervention(
    SourcelessIntervention,
    TrainableIntervention, 
    DistributedRepresentationIntervention
):
    """
    LoRA(h') = h' + BAh
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs, keep_last_dim=True)
        self.r = kwargs["low_rank_dimension"]
        self.lora_alpha = kwargs["alpha"] if "alpha" in kwargs else kwargs["low_rank_dimension"]
        if "dropout" in kwargs and kwargs["dropout"] > 0.0:
            self.lora_dropout = nn.Dropout(p=kwargs["dropout"])
        else:
            self.lora_dropout = lambda x: x

        # Actual trainable parameters
        self.lora_A = nn.Parameter(torch.zeros(self.embed_dim, kwargs["low_rank_dimension"]))
        self.lora_B = nn.Parameter(torch.zeros(kwargs["low_rank_dimension"], 3 * self.embed_dim))
        # self.lora_B = nn.Parameter(torch.zeros(kwargs["low_rank_dimension"], self.embed_dim))

        # initialize A the same way as the default for nn.Linear and B to zero
        nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
        nn.init.zeros_(self.lora_B)
        self.lora_A = nn.Parameter(self.lora_A.to(torch.bfloat16))
        self.lora_B = nn.Parameter(self.lora_B.to(torch.bfloat16))

        mag = None
        if "mag" in kwargs:
            mag = kwargs["mag"].copy()
            del kwargs["mag"]
        self.mag = torch.tensor(mag).to(device) if mag is not None else torch.ones(1).to(device)
        self.mag = self.mag.to(torch.bfloat16)
        self.register_buffer('cumulative_flops', torch.tensor(0))
            
    def calculate_flops(self, input_shape):
        """
        Calculates the FLOPs for the LoraIntervention.

        Args:
            input_shape (tuple): The shape of the input tensor. Expects (batch_size, seq_length, embed_dim).

        Returns:
            total_flops (int): Total FLOPs for the forward pass.
        """
        batch_size, seq_length, embed_dim = input_shape
        # print(batch_size, seq_length, embed_dim, self.r)

        # FLOPs for first matrix multiplication: (batch_size * seq_length, embed_dim) @ (embed_dim, low_rank_dimension)
        flops_A = 2 * batch_size * seq_length * embed_dim * self.r

        # FLOPs for second matrix multiplication: (batch_size * seq_length, low_rank_dimension) @ (low_rank_dimension, 3 * embed_dim)
        flops_B = 2 * batch_size * seq_length * self.r * (3 * embed_dim)
        # flops_B = 2 * batch_size * seq_length * self.r * (embed_dim)

        # FLOPs for addition: (batch_size * seq_length * embed_dim)
        flops_add = batch_size * seq_length * embed_dim

        # Total FLOPs
        total_flops = flops_A + flops_B + flops_add

        return total_flops

    def forward(
        self, base, source=None, subspaces=None, **kwargs
    ):
        original_input = kwargs["_pyvene_model_input_args"][0]
        
        # Calculate FLOPs for the current forward pass
        flops = self.calculate_flops(original_input.shape)
        
        # Optionally store FLOPs for logging or later use
        if hasattr(self, 'cumulative_flops'):
            self.cumulative_flops += flops
        else:
            self.cumulative_flops = flops

        return base + self.mag * self.lora_dropout(original_input) @ self.lora_A @ self.lora_B


#### Load LoRA config

In [11]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)


In [12]:
layers = [15]

# get reft model
reft_config = ReftConfig(representations=
    [{
            "layer": l, "component": "transformer.h.15.attn.c_attn.output",
            "low_rank_dimension": low_rank,
            "intervention": LoraIntervention(
                embed_dim=model.config.hidden_size, low_rank_dimension=low_rank,
                dtype=torch.bfloat16, 
                init_orth=True,
            )
        } for l in layers]
)
reft_model = get_reft_model(model, reft_config, set_device=False)
reft_model.set_device(device)
print(reft_model.get_device())
reft_model.print_trainable_parameters()

cuda:0
trainable intervention params: 20,480 || trainable model params: 0
model params: 774,030,080 || trainable%: 0.002645892004610467


In [13]:
data_ret = make_all_positions_unsupervised_data_module(tokenizer, model, raw_dataset, num_interventions=len(layers), nonstop=False)

In [14]:
train_dataset = data_ret["train_dataset"]
data_collator = data_ret["data_collator"]

#### Training!

In [15]:
from torch.utils.tensorboard import SummaryWriter

from transformers import TrainerCallback

class FlopsLoggingCallback(TrainerCallback):
    def __init__(self):
        self.writer = SummaryWriter()  # Initialize SummaryWriter

    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero and self.writer is not None:
            if "loss" in logs:
                # Calculate total_flops from your model
                total_flops = 0
                # print(kwargs['model'].interventions)
                for k, v in kwargs['model'].interventions.items():
                    if isinstance(v[0], LoraIntervention):
                        total_flops = v[0].cumulative_flops

                # Log FLOPs to TensorBoard
                self.writer.add_scalar('FLOPs', total_flops, global_step=state.global_step)
                print(f"Global Step: {state.global_step}, Calculated FLOPs: {total_flops}")


In [16]:
# train
training_args = transformers.TrainingArguments(
    num_train_epochs=3.0, output_dir="./results_reft", learning_rate=1e-3, report_to=["wandb"],
    per_device_train_batch_size=train_batch_size, logging_steps=300, bf16=True,
    warmup_ratio=0.06,
)
trainer = ReftTrainerForCausalLM(
    model=reft_model, tokenizer=tokenizer, args=training_args, 
    train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator,
    callbacks=[FlopsLoggingCallback()]
)
trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: peterzw494 (peterwz). Use `wandb login --relogin` to force relogin


Step,Training Loss
300,3.993400
600,3.914300
900,3.852800
1200,3.839800
1500,3.836200
1800,3.818200
2100,3.854900
2400,3.815300
2700,3.785800
3000,3.790800


Global Step: 300, Calculated FLOPs: 1851379200
Directory './results_reft/checkpoint-500/intervenable_model' already exists.
Global Step: 600, Calculated FLOPs: 3721681920
Global Step: 900, Calculated FLOPs: 5717775360
Directory './results_reft/checkpoint-1000/intervenable_model' already exists.
Global Step: 1200, Calculated FLOPs: 7702464000
Global Step: 1500, Calculated FLOPs: 9602165760
Directory './results_reft/checkpoint-1500/intervenable_model' already exists.
Global Step: 1800, Calculated FLOPs: 11534476800
Directory './results_reft/checkpoint-2000/intervenable_model' already exists.
Global Step: 2100, Calculated FLOPs: 13440683520
Global Step: 2400, Calculated FLOPs: 15347397120
Directory './results_reft/checkpoint-2500/intervenable_model' already exists.
Global Step: 2700, Calculated FLOPs: 17258672640
Global Step: 3000, Calculated FLOPs: 19155333120
Directory './results_reft/checkpoint-3000/intervenable_model' already exists.


TrainOutput(global_step=3000, training_loss=3.8501295166015623, metrics={'train_runtime': 166.4266, 'train_samples_per_second': 36.052, 'train_steps_per_second': 18.026, 'total_flos': 0.0, 'train_loss': 3.8501295166015623, 'epoch': 3.0})

#### Check the Background GPT-2 toxicity and perplexity

Let's checkout the background GPT-2 performance!

In [17]:
reft_model.eval()
reft_model.training = False
ret = {}

In [18]:
reft_model.interventions.keys()

dict_keys(['comp.transformer.h.15.attn.c_attn.output.unit.pos.nunit.1#0'])

In [19]:
for i in layers:
    key = 'comp.transformer.h.15.attn.c_attn.output.unit.pos.nunit.1#0'
    reft_model.interventions[key][0].mag = 0.0 * torch.ones(1).to(device).to(torch.bfloat16)

tox_mean, tox_std = generate_toxicity(layers)
ppl = calculate_perplexity(layers)
ret["lora_pyreft_0"] = (tox_mean, tox_std, ppl)

  0%|                                                                                     | 0/16 [00:00<?, ?it/s]/nlp/scr/peterwz/miniconda3/envs/peterwz-comp/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████| 16/16 [00:44<00:00,  2.76s/it]
/nlp/scr/peterwz/miniconda3/envs/peterwz-comp/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0.07239165349199084 0.20904985270364193
haha 287644
1


100%|█████████████████████████████████████████████████████████████████████████▋| 560/562 [00:33<00:00, 16.55it/s]


tensor(16.4457, device='cuda:0')


#### Check the "toxic" intervention
Let's check the learned "toxic" intervention.

In [20]:
for i in layers:
    key = 'comp.transformer.h.15.attn.c_attn.output.unit.pos.nunit.1#0'
    reft_model.interventions[key][0].mag = 1.0 * torch.ones(1).to(device).to(torch.bfloat16)

tox_mean, tox_std = generate_toxicity(layers)
ppl = calculate_perplexity(layers)
ret["lora_pyreft_1"] = (tox_mean, tox_std, ppl)

100%|████████████████████████████████████████████████████████████████████████████| 16/16 [00:43<00:00,  2.72s/it]


0.2356256846955489 0.36062346937609063
haha 287644
1


100%|█████████████████████████████████████████████████████████████████████████▋| 560/562 [00:34<00:00, 16.28it/s]


tensor(18.3466, device='cuda:0')


#### Check the "Untoxicfied" GPT-2
Let's reverse that intervention and see the resulted model.

In [21]:
for i in layers:
    key = 'comp.transformer.h.15.attn.c_attn.output.unit.pos.nunit.1#0'
    reft_model.interventions[key][0].mag = -1.0 * torch.ones(1).to(device).to(torch.bfloat16)

tox_mean, tox_std = generate_toxicity(layers)
ppl = calculate_perplexity(layers)
ret["lora_pyreft_-1"] = (tox_mean, tox_std, ppl)

100%|████████████████████████████████████████████████████████████████████████████| 16/16 [00:43<00:00,  2.74s/it]


0.02782047360364004 0.11401897784032253
haha 287644
1


100%|█████████████████████████████████████████████████████████████████████████▋| 560/562 [00:34<00:00, 16.25it/s]


tensor(18.3431, device='cuda:0')


We can see that 

- We can implement LoRA with the Pyvene/Pyreft library
- We reproduced the positive/negative (unlearning) experiment in the LoRA operator paper
- However, LoRA's perplexity increased significantly after fine-tuning, on both the positive direction and the negative direction

<a id="section-3"></a>
### 3. ReFT on all positions

We have tried LoRA on all positions. What about the performance of ReFT?

In [22]:
class SubloreftIntervention(LoreftIntervention):
    """
    This is a LoReFT that supports subspace interventions with coefficients!
    """
    def __init__(self, **kwargs):
        subspace_coeff = None
        # Subspace coefficients are the coefficients applied to each subspace.
        # When `subspace_coeff` is a ones tensor, this intervention is the same as a loreft intervention with subspaces
        # When `subspace_coeff` is a negative-ones tensor, this intervention is the negation of the loreft intervention
        # There is no intervention when `subspace_coeff` is zero.
        if "subspace_coeff" in kwargs:
            subspace_coeff = kwargs["subspace_coeff"].copy()
            del kwargs["subspace_coeff"]
        self.subspace_coeff = torch.tensor(subspace_coeff).to(device) if subspace_coeff is not None else torch.ones(kwargs["low_rank_dimension"]).to(device)
        print(kwargs)
        super().__init__(**kwargs)
        self.register_buffer('cumulative_flops', torch.tensor(0))
            
    def forward(
        self, base, source=None, subspaces=None, **kwargs
    ):
        assert subspaces is not None
        output = []
        total_flops = 0

        rotated_base = self.rotate_layer(base)
        # print(base.shape)
        total_flops += 2 * base.shape[0] * base.shape[1] * base.shape[2] * self.rotate_layer.weight.shape[1]

        diff = self.act_fn(self.learned_source(base)) - rotated_base
        total_flops += 2 * base.shape[0] * base.shape[1] * base.shape[2] * self.learned_source.weight.shape[0]  # Matmul
        total_flops += base.shape[0] * base.shape[1] * self.learned_source.weight.shape[0]  # Bias addition

        
        # print(base.shape[0], base.shape[1], base.shape[2], self.learned_source.weight.shape[0])
        batched_subspace = []
        batched_weights = []
        
        for example_i in range(len(diff)):
            # Apply potential negations/coefficients here
            # print(diff.shape, base.shape)
            # print(subspaces)
            LHS = (diff[example_i, :, subspaces[example_i]]) * self.subspace_coeff[subspaces[example_i]]
            RHS = self.rotate_layer.weight[..., subspaces[example_i]] 
            RHS = RHS.T
            batched_subspace += [LHS]
            batched_weights += [RHS]
            # FLOPs for LHS multiplication (assuming element-wise)
            flops_elementwise = LHS.numel()
            # print(flops_elementwise)
            total_flops += flops_elementwise


        batched_subspace = torch.stack(batched_subspace, dim=0)
        batched_weights = torch.stack(batched_weights, dim=0)
        
        output = base + torch.bmm(batched_subspace, batched_weights)
        flops_batched_mm = 2 * batched_subspace.shape[0] * batched_subspace.shape[1] * batched_subspace.shape[2] * batched_weights.shape[2]
        total_flops += flops_batched_mm
        # print(batched_subspace.shape, batched_weights.shape)
        total_flops += output.numel()

        self.cumulative_flops += total_flops


        return self.dropout(output.to(base.dtype))

In [23]:
layers = [15]

# get reft model
reft_config = ReftConfig(representations=
    [{
            "layer": l, "component": "block_output",
            "low_rank_dimension": low_rank,
            "intervention": SubloreftIntervention(
                embed_dim=model.config.hidden_size, low_rank_dimension=low_rank,
                dtype=torch.bfloat16, 
                init_orth=True,
            )
        } for l in layers]
)
reft_model = get_reft_model(model, reft_config, set_device=False)
reft_model.train()
reft_model.training = True
reft_model.set_device(device)
print(reft_model.get_device())
reft_model.print_trainable_parameters()

{'embed_dim': 1280, 'low_rank_dimension': 4, 'dtype': torch.bfloat16, 'init_orth': True}
cuda:0
trainable intervention params: 10,244 || trainable model params: 0
model params: 774,030,080 || trainable%: 0.0013234627780873839


In [24]:
from torch.utils.tensorboard import SummaryWriter

from transformers import TrainerCallback

class FlopsLoggingCallback(TrainerCallback):
    def __init__(self):
        self.writer = SummaryWriter()  # Initialize SummaryWriter

    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero and self.writer is not None:
            if "loss" in logs:
                # Calculate total_flops from your model
                total_flops = 0
                # print(kwargs['model'].interventions)
                for k, v in kwargs['model'].interventions.items():
                    if isinstance(v[0], SubloreftIntervention):
                        total_flops = v[0].cumulative_flops

                # Log FLOPs to TensorBoard
                self.writer.add_scalar('FLOPs', total_flops, global_step=state.global_step)
                print(f"Global Step: {state.global_step}, Calculated FLOPs: {total_flops}")

In [25]:
# train
training_args = transformers.TrainingArguments(
    num_train_epochs=3.0, output_dir="./results_reft", learning_rate=1e-3, report_to=["wandb"],
    per_device_train_batch_size=train_batch_size, logging_steps=300, bf16=True,
    warmup_ratio=0.06,
)
trainer = ReftTrainerForCausalLM(
    model=reft_model, tokenizer=tokenizer, args=training_args, 
    train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator,
    callbacks=[FlopsLoggingCallback()]
)
trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
300,3.964000
600,3.877600
900,3.805000
1200,3.785600
1500,3.781100
1800,3.758800
2100,3.782300
2400,3.742300
2700,3.716600
3000,3.720500


Global Step: 300, Calculated FLOPs: 1402910640
Directory './results_reft/checkpoint-500/intervenable_model' already exists.
Global Step: 600, Calculated FLOPs: 2820160864
Global Step: 900, Calculated FLOPs: 4332730912
Directory './results_reft/checkpoint-1000/intervenable_model' already exists.
Global Step: 1200, Calculated FLOPs: 5836658800
Global Step: 1500, Calculated FLOPs: 7276186592
Directory './results_reft/checkpoint-1500/intervenable_model' already exists.
Global Step: 1800, Calculated FLOPs: 8740424560
Directory './results_reft/checkpoint-2000/intervenable_model' already exists.
Global Step: 2100, Calculated FLOPs: 10184881584
Global Step: 2400, Calculated FLOPs: 11629722704
Directory './results_reft/checkpoint-2500/intervenable_model' already exists.
Global Step: 2700, Calculated FLOPs: 13078020688
Global Step: 3000, Calculated FLOPs: 14515243904
Directory './results_reft/checkpoint-3000/intervenable_model' already exists.


TrainOutput(global_step=3000, training_loss=3.7933878173828126, metrics={'train_runtime': 178.2313, 'train_samples_per_second': 33.664, 'train_steps_per_second': 16.832, 'total_flos': 0.0, 'train_loss': 3.7933878173828126, 'epoch': 3.0})

In [26]:
reft_model.eval()
reft_model.training = False

In [27]:
reft_model.interventions.keys()

dict_keys(['layer.15.comp.block_output.unit.pos.nunit.1#0'])

In [28]:
for i in layers:
    key = 'layer.' + str(i) + '.comp.block_output.unit.pos.nunit.1#0'
    reft_model.interventions[key][0].subspace_coeff = 0.0 * torch.ones(low_rank).to(device)

tox_mean, tox_std = generate_toxicity(layers)
ppl = calculate_perplexity(layers)
ret["reft_all_0"] = (tox_mean, tox_std, ppl)

  0%|                                                                                     | 0/16 [00:00<?, ?it/s]/nlp/scr/peterwz/miniconda3/envs/peterwz-comp/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████| 16/16 [01:29<00:00,  5.61s/it]
/nlp/scr/peterwz/miniconda3/envs/peterwz-comp/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0.07239165349199084 0.20904985270364193
haha 287644
1


100%|█████████████████████████████████████████████████████████████████████████▋| 560/562 [00:33<00:00, 16.74it/s]

tensor(16.4457, device='cuda:0')


In [29]:
for i in layers:
    key = 'layer.' + str(i) + '.comp.block_output.unit.pos.nunit.1#0'
    reft_model.interventions[key][0].subspace_coeff = 1.0 * torch.ones(low_rank).to(device)

tox_mean, tox_std = generate_toxicity(layers)
ppl = calculate_perplexity(layers)
ret["reft_all_1"] = (tox_mean, tox_std, ppl)

100%|████████████████████████████████████████████████████████████████████████████| 16/16 [01:29<00:00,  5.57s/it]


0.2254427645853525 0.34547832646255394
haha 287644
1


100%|█████████████████████████████████████████████████████████████████████████▋| 560/562 [00:33<00:00, 16.61it/s]

tensor(16.6782, device='cuda:0')


In [30]:
for i in layers:
    key = 'layer.' + str(i) + '.comp.block_output.unit.pos.nunit.1#0'
    reft_model.interventions[key][0].subspace_coeff = -1.0 * torch.ones(low_rank).to(device)

tox_mean, tox_std = generate_toxicity(layers)
ppl = calculate_perplexity(layers)
ret["reft_all_-1"] = (tox_mean, tox_std, ppl)

100%|████████████████████████████████████████████████████████████████████████████| 16/16 [01:29<00:00,  5.59s/it]


0.0213432774300486 0.09923869876326916
haha 287644
1


100%|█████████████████████████████████████████████████████████████████████████▋| 560/562 [00:33<00:00, 16.60it/s]

tensor(17.2887, device='cuda:0')


We can see that on the same task, ReFT intervening on all positions

- Took less training flops than LoRA (74%)
- Achieved similar performance on both the positive direction and the negative direction (toxicity)
- Preserved better generation fluency than LoRA (perplexity).


<a id="section-4"></a>
### 4. ReFT on only the first position
What about we do ReFT, but only on the first position? This is very intriguing - we only need to intervene on a single position on a single layer of the language model's representations, and we can meaningfully steer the model's behavior towards a direction, positively or negatively.

In [31]:
layers = [15]

# get reft model
reft_config = ReftConfig(representations=
    [{
            "layer": l, "component": "block_output",
            "low_rank_dimension": low_rank,
            "intervention": SubloreftIntervention(
                embed_dim=model.config.hidden_size, low_rank_dimension=low_rank,
                dtype=torch.bfloat16, 
                init_orth=True,
            )
        } for l in layers]
)
reft_model = get_reft_model(model, reft_config, set_device=False)
reft_model.train()
reft_model.training = True
reft_model.set_device(device)
print(reft_model.get_device())
reft_model.print_trainable_parameters()

{'embed_dim': 1280, 'low_rank_dimension': 4, 'dtype': torch.bfloat16, 'init_orth': True}
cuda:0
trainable intervention params: 10,244 || trainable model params: 0
model params: 774,030,080 || trainable%: 0.0013234627780873839


In [34]:
ret_dict = make_all_positions_unsupervised_data_module(tokenizer, model, raw_dataset, num_interventions=len(layers), nonstop=False,
                                                 intervene_on_all=False)
train_dataset = ret_dict["train_dataset"]
data_collator = ret_dict["data_collator"]

In [35]:
# train
training_args = transformers.TrainingArguments(
    num_train_epochs=3.0, output_dir="./results_reft", learning_rate=1e-3, report_to=["wandb"],
    per_device_train_batch_size=train_batch_size, logging_steps=300, bf16=True,
    warmup_ratio=0.06,
)
trainer = ReftTrainerForCausalLM(
    model=reft_model, tokenizer=tokenizer, args=training_args, 
    train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator,
    callbacks=[FlopsLoggingCallback()])
trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
300,4.016500
600,3.974500
900,3.924700
1200,3.910700
1500,3.918300
1800,3.902800
2100,3.927200
2400,3.903800
2700,3.868600
3000,3.873400


Global Step: 300, Calculated FLOPs: 19204800
Directory './results_reft/checkpoint-500/intervenable_model' already exists.
Global Step: 600, Calculated FLOPs: 38409600
Global Step: 900, Calculated FLOPs: 57614400
Directory './results_reft/checkpoint-1000/intervenable_model' already exists.
Global Step: 1200, Calculated FLOPs: 76819200
Global Step: 1500, Calculated FLOPs: 96024000
Directory './results_reft/checkpoint-1500/intervenable_model' already exists.
Global Step: 1800, Calculated FLOPs: 115228800
Directory './results_reft/checkpoint-2000/intervenable_model' already exists.
Global Step: 2100, Calculated FLOPs: 134433600
Global Step: 2400, Calculated FLOPs: 153638400
Directory './results_reft/checkpoint-2500/intervenable_model' already exists.
Global Step: 2700, Calculated FLOPs: 172843200
Global Step: 3000, Calculated FLOPs: 192048000
Directory './results_reft/checkpoint-3000/intervenable_model' already exists.


TrainOutput(global_step=3000, training_loss=3.922053507486979, metrics={'train_runtime': 179.401, 'train_samples_per_second': 33.445, 'train_steps_per_second': 16.722, 'total_flos': 0.0, 'train_loss': 3.922053507486979, 'epoch': 3.0})

In [36]:
reft_model.eval()
reft_model.training = False

In [37]:
for i in layers:
    key = 'layer.' + str(i) + '.comp.block_output.unit.pos.nunit.1#0'
    reft_model.interventions[key][0].subspace_coeff = 0.0 * torch.ones(low_rank).to(device)

tox_mean, tox_std = generate_toxicity(layers, intervene_on_all=False)
ppl = calculate_perplexity(layers, intervene_on_all=False)
ret["reft_f1_0"] = (tox_mean, tox_std, ppl)

  0%|                                                                                     | 0/16 [00:00<?, ?it/s]/nlp/scr/peterwz/miniconda3/envs/peterwz-comp/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████| 16/16 [00:42<00:00,  2.66s/it]
/nlp/scr/peterwz/miniconda3/envs/peterwz-comp/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0.07239165349199084 0.20904985270364193
haha 287644
1


100%|█████████████████████████████████████████████████████████████████████████▋| 560/562 [00:34<00:00, 16.43it/s]

tensor(16.4457, device='cuda:0')


In [38]:
for i in layers:
    key = 'layer.' + str(i) + '.comp.block_output.unit.pos.nunit.1#0'
    reft_model.interventions[key][0].subspace_coeff = 1.0 * torch.ones(low_rank).to(device)

tox_mean, tox_std = generate_toxicity(layers, intervene_on_all=False)
ppl = calculate_perplexity(layers, intervene_on_all=False)
ret["reft_f1_1"] = (tox_mean, tox_std, ppl)

100%|████████████████████████████████████████████████████████████████████████████| 16/16 [00:43<00:00,  2.70s/it]


0.27369099783618367 0.37188162905204397
haha 287644
1


100%|█████████████████████████████████████████████████████████████████████████▋| 560/562 [00:34<00:00, 16.18it/s]


tensor(17.0535, device='cuda:0')


In [39]:
for i in layers:
    key = 'layer.' + str(i) + '.comp.block_output.unit.pos.nunit.1#0'
    reft_model.interventions[key][0].subspace_coeff = -1.0 * torch.ones(low_rank).to(device)

tox_mean, tox_std = generate_toxicity(layers, intervene_on_all=False)
ppl = calculate_perplexity(layers, intervene_on_all=False)
ret["reft_f1_-1"] = (tox_mean, tox_std, ppl)

100%|████████████████████████████████████████████████████████████████████████████| 16/16 [00:42<00:00,  2.65s/it]


0.019930323498101643 0.09382162088810096
haha 287644
1


100%|█████████████████████████████████████████████████████████████████████████▋| 560/562 [00:34<00:00, 16.15it/s]

tensor(16.8638, device='cuda:0')


We can see that ReFT on a single position (first position of the prompt) took much less flops (1%) than both LoRA and ReFT on all positions. This is because, here the sequence length fed into the intervention is 1, whereas before the sequence length was 512 (or longer).

However, we can see that ReFT on a single position achieved much better positive (and negative) performance on the toxicity unlearning task, while maintaining a lower level of perplexity!

<a id="appendix-1"></a>
### Appendix 1. ReFT on the same model position of LoRA

We cannot apply LoReFT intervention on `15.attn.c_attn` in the same way as LoRA. Assume that `h' = c_attn(h)`, LoRA applies the intervention as `h' = h' + BAh`. It requires `h` as the input, and the intervention is added on `h'`. However, we can create a similar intervention with ReFT as `h = h + LoReFT(h)`. This is the closest setup we can implement for LoReFT comparing with LoRA.

In [40]:
layers = [15]

# get reft model
reft_config = ReftConfig(representations=
    [{
            "layer": l, "component": "transformer.h.15.attn.c_attn.input",
            "low_rank_dimension": low_rank,
            "intervention": SubloreftIntervention(
                embed_dim=model.config.hidden_size, low_rank_dimension=low_rank,
                dtype=torch.bfloat16, 
                init_orth=True,
            )
        } for l in layers]
)
reft_model = get_reft_model(model, reft_config, set_device=False)
reft_model.train()
reft_model.training = True
reft_model.set_device(device)
print(reft_model.get_device())
reft_model.print_trainable_parameters()

{'embed_dim': 1280, 'low_rank_dimension': 4, 'dtype': torch.bfloat16, 'init_orth': True}
cuda:0
trainable intervention params: 10,244 || trainable model params: 0
model params: 774,030,080 || trainable%: 0.0013234627780873839


In [41]:
from torch.utils.tensorboard import SummaryWriter

from transformers import TrainerCallback

class FlopsLoggingCallback(TrainerCallback):
    def __init__(self):
        self.writer = SummaryWriter()  # Initialize SummaryWriter

    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero and self.writer is not None:
            if "loss" in logs:
                # Calculate total_flops from your model
                total_flops = 0
                # print(kwargs['model'].interventions)
                for k, v in kwargs['model'].interventions.items():
                    if isinstance(v[0], SubloreftIntervention):
                        total_flops = v[0].cumulative_flops

                # Log FLOPs to TensorBoard
                self.writer.add_scalar('FLOPs', total_flops, global_step=state.global_step)
                print(f"Global Step: {state.global_step}, Calculated FLOPs: {total_flops}")

In [42]:
# train
training_args = transformers.TrainingArguments(
    num_train_epochs=3.0, output_dir="./results_reft", learning_rate=1e-3, report_to=["wandb"],
    per_device_train_batch_size=train_batch_size, logging_steps=300, bf16=True,
    warmup_ratio=0.06,
)
trainer = ReftTrainerForCausalLM(
    model=reft_model, tokenizer=tokenizer, args=training_args, 
    train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator,
    callbacks=[FlopsLoggingCallback()]
)
trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
300,3.989000
600,3.958300
900,3.913600
1200,3.906500
1500,3.910200
1800,3.895600
2100,3.935700
2400,3.898600
2700,3.870100
3000,3.879000


Global Step: 300, Calculated FLOPs: 19204800
Directory './results_reft/checkpoint-500/intervenable_model' already exists.
Global Step: 600, Calculated FLOPs: 38409600
Global Step: 900, Calculated FLOPs: 57614400
Directory './results_reft/checkpoint-1000/intervenable_model' already exists.
Global Step: 1200, Calculated FLOPs: 76819200
Global Step: 1500, Calculated FLOPs: 96024000
Directory './results_reft/checkpoint-1500/intervenable_model' already exists.
Global Step: 1800, Calculated FLOPs: 115228800
Directory './results_reft/checkpoint-2000/intervenable_model' already exists.
Global Step: 2100, Calculated FLOPs: 134433600
Global Step: 2400, Calculated FLOPs: 153638400
Directory './results_reft/checkpoint-2500/intervenable_model' already exists.
Global Step: 2700, Calculated FLOPs: 172843200
Global Step: 3000, Calculated FLOPs: 192048000
Directory './results_reft/checkpoint-3000/intervenable_model' already exists.


TrainOutput(global_step=3000, training_loss=3.91566259765625, metrics={'train_runtime': 183.5492, 'train_samples_per_second': 32.689, 'train_steps_per_second': 16.344, 'total_flos': 0.0, 'train_loss': 3.91566259765625, 'epoch': 3.0})

In [43]:
reft_model.eval()
reft_model.training = False

In [44]:
reft_model.interventions.keys()

dict_keys(['comp.transformer.h.15.attn.c_attn.input.unit.pos.nunit.1#0'])

In [45]:
for i in layers:
    key = 'comp.transformer.h.' + str(i) + '.attn.c_attn.input.unit.pos.nunit.1#0'
    reft_model.interventions[key][0].subspace_coeff = 0.0 * torch.ones(low_rank).to(device)

tox_mean, tox_std = generate_toxicity(layers, intervene_on_all=False)
ppl = calculate_perplexity(layers, intervene_on_all=False)
ret["reft_f1_cattn_0"] = (tox_mean, tox_std, ppl)

  0%|                                                                                     | 0/16 [00:00<?, ?it/s]/nlp/scr/peterwz/miniconda3/envs/peterwz-comp/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████| 16/16 [00:42<00:00,  2.68s/it]
/nlp/scr/peterwz/miniconda3/envs/peterwz-comp/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0.07239165349199084 0.20904985270364193
haha 287644
1


100%|█████████████████████████████████████████████████████████████████████████▋| 560/562 [00:33<00:00, 16.58it/s]

tensor(16.4457, device='cuda:0')


In [46]:
for i in layers:
    key = 'comp.transformer.h.' + str(i) + '.attn.c_attn.input.unit.pos.nunit.1#0'
    reft_model.interventions[key][0].subspace_coeff = 1.0 * torch.ones(low_rank).to(device)

tox_mean, tox_std = generate_toxicity(layers, intervene_on_all=False)
ppl = calculate_perplexity(layers, intervene_on_all=False)
ret["reft_f1_cattn_1"] = (tox_mean, tox_std, ppl)

100%|████████████████████████████████████████████████████████████████████████████| 16/16 [00:42<00:00,  2.68s/it]


0.22378594287988562 0.3495321010104451
haha 287644
1


100%|█████████████████████████████████████████████████████████████████████████▋| 560/562 [00:34<00:00, 16.31it/s]

tensor(16.7241, device='cuda:0')


In [47]:
for i in layers:
    key = 'comp.transformer.h.' + str(i) + '.attn.c_attn.input.unit.pos.nunit.1#0'
    reft_model.interventions[key][0].subspace_coeff = -1.0 * torch.ones(low_rank).to(device)

tox_mean, tox_std = generate_toxicity(layers, intervene_on_all=False)
ppl = calculate_perplexity(layers, intervene_on_all=False)
ret["reft_f1_cattn_-1"] = (tox_mean, tox_std, ppl)

100%|████████████████████████████████████████████████████████████████████████████| 16/16 [00:45<00:00,  2.83s/it]


0.0489121805875925 0.15722763116930472
haha 287644
1


100%|█████████████████████████████████████████████████████████████████████████▋| 560/562 [00:34<00:00, 16.37it/s]

tensor(18.1124, device='cuda:0')


<a id="appendix-2"></a>
### Appendix 2. LoRA on Hugging Face PEFT library

We implemented LoRA via the Pyvene/Pyreft library in section 2. Below we reproduce the same experiment, but on Hugging Face's PEFT library. PEFT LoRA implementation is different from PyReFT LoRA implementation, but we can see the same increase of perplexity compared with ReFT.

In [48]:
from peft import LoraConfig


lora_config = LoraConfig(
    r=low_rank,  # rank of the adaptation matrix
    lora_alpha=low_rank,  # alpha parameter for scaling
    lora_dropout=0,  # dropout rate for the adaptation matrix
    # target_modules=['c_attn', "mlp.c_fc", "mlp.c_proj"],  # target attention matrices
    target_modules=['15.attn.c_attn'],  # target attention matrices
    init_lora_weights=True,  # initialization of weights
    # lora_composition_mode='add',  # composition mode for the adaptation
    # self_attn_lora=True,  # apply LoRA to self-attention
    # intermediate_lora=False,  # do not apply LoRA to intermediate layers
    # output_lora=False,  # do not apply LoRA to output layers
    # use_gating=False,  # do not use gating mechanism
)
from peft import PeftModel

# Wrap the base model with the PeftModel
peft_model = PeftModel(model, lora_config)
peft_model.to(device)
peft_model.train()
peft_model.training = True

/nlp/scr/peterwz/miniconda3/envs/peterwz-comp/lib/python3.11/site-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [49]:
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results_peft',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    learning_rate=1e-3,
    warmup_ratio=0.06,
    bf16 = True,
    logging_steps=50,
    report_to=["wandb"],
)
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
    padding="longest"
)

In [50]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    # callbacks=[ProfilingCallback],
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [51]:
trainer.train()

Step,Training Loss
50,4.117200
100,4.222500
150,4.012700
200,3.931200
250,3.798800
300,3.843300
350,3.863400
400,3.917600
450,3.925700
500,3.979200


TrainOutput(global_step=3000, training_loss=3.83700542195638, metrics={'train_runtime': 175.7107, 'train_samples_per_second': 34.147, 'train_steps_per_second': 17.074, 'total_flos': 1929813248286720.0, 'train_loss': 3.83700542195638, 'epoch': 3.0})

In [52]:
def peft_generate_toxicity():
    # instruction = " "
    
    prompt = [instruction] * gen_batch_size
    prompt = tokenizer(prompt, return_tensors="pt")
    # prompt = tokenizer.encode(instruction, return_tensors="pt")

    prompt = prompt.to(device)
    # print(prompt)
    # print(prompt.shape)
        
    toxicities = []
    pbar = tqdm(range(1024 // gen_batch_size))
    # pbar = tqdm(range(16))
    generated_texts = []
    for _ in pbar:
        #peft_model
        #prompt["input_ids"]
        outputs = peft_model.generate(prompt["input_ids"], max_new_tokens=128,  do_sample=True,
                                no_repeat_ngram_size=5, 
                                repetition_penalty=1.1,
                                # top_k = 50,
                                eos_token_id=tokenizer.eos_token_id, early_stopping=True,
                                pad_token_id=tokenizer.eos_token_id)

        # Decode and print the generated text
        generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        
        # print(generated_text)
        # print("====")
        generated_text = [t[len(instruction):] for t in generated_text]
        # print(generated_text)
        # print("****")
        generated_texts += generated_text

        # print(Detoxify("original").predict(generated_text))
    # print(generated_texts[0:100:10]) # [:100]
    toxicity = Detoxify("original", device=device).predict(generated_texts)["toxicity"]
    # toxicities.append(toxicity)
    # pbar.set_description("Toxicity: " + str(toxicity))
    mean = np.mean(toxicity)
    std = np.std(toxicity)
    print(mean, std)
    return mean, std

In [53]:
def peft_calculate_perplexity():
    
    max_length = peft_model.config.n_positions
    stride = 512
    seq_len = encodings.input_ids.size(1)
    print('haha',seq_len)
    nlls = []
    prev_end_loc = 0
    print(torch.cuda.device_count())
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone().detach()
        target_ids[:, :-trg_len] = -100
    
        with torch.no_grad():

            outputs = peft_model(input_ids, labels=target_ids)
    
            # loss is calculated using CrossEntropyLoss which averages over input tokens.
            # Multiply it with trg_len to get the summation instead of average.
            # We will take average over all the tokens to get the true average
            # in the last step of this example.
            # print(input_ids)
            # print("===")
            # print(outputs.loss)
            # print("***")
            # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
            neg_log_likelihood = outputs.loss * trg_len
    
        nlls.append(neg_log_likelihood)
    
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break
    
    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    print(ppl)
    return ppl.item()

In [54]:
tox_mean, tox_std = peft_generate_toxicity()
ppl = peft_calculate_perplexity()
ret["lora_peft_1"] = (tox_mean, tox_std, ppl)

  0%|                                                                                     | 0/16 [00:00<?, ?it/s]/nlp/scr/peterwz/miniconda3/envs/peterwz-comp/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████| 16/16 [01:06<00:00,  4.15s/it]
/nlp/scr/peterwz/miniconda3/envs/peterwz-comp/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0.19926864944710587 0.3294361210167149
haha 287644
1


100%|█████████████████████████████████████████████████████████████████████████▋| 560/562 [00:34<00:00, 16.36it/s]


tensor(18.4564, device='cuda:0')


In [55]:
def adapter_scaling(
    model,
    adapter_config,
    scale
    ):
    # config = AutoConfig.from_pretrained(
    #     model_name_or_path,
    # )
    # model = AutoAdapterModel.from_pretrained(
    #     model_name_or_path,
    #     cache_dir=cache_dir,
    #     config=config
    # )
    # model=GPT2LMHeadModel.from_pretrained(
    #     model_name_or_path,
    #     cache_dir=cache_dir,
    #     config=config
    # )

    state_dict = model.state_dict()
    merged_keys = [mk for mk in state_dict.keys() if  ("adapters" in mk) or ("lora" in mk)]
    # breakpoint()
    if adapter_config=="lora":
        # neg_dict = {k:-v for k,v in state_dict.items() if "lora_A" in k}
        neg_dict = {k:scale*v for k,v in state_dict.items() if "lora_A" in k}

    # ia3 (h+l*delta_h)-(h+delta_h)=(l-1)*delta_h h+delta_h-(l-1)*delta_h=h+(2-l)*delta_h
    elif adapter_config=="ia3":
        # neg_dict = {k:(torch.ones(v.shape)*2-v) for k,v in state_dict.items() if "lora" in k}
        neg_dict = {k:(torch.ones(v.shape)*(1-scale)+scale*v) for k,v in state_dict.items() if "lora" in k}

    state_dict.update(neg_dict)
    model.load_state_dict(state_dict)
    # model.set_active_adapters(["civil_comments"])
    # model.save_all_adapters(save_path)

    return model

In [56]:
peft_model = adapter_scaling(peft_model, "lora", -1.0)
tox_mean, tox_std = peft_generate_toxicity()
ppl = peft_calculate_perplexity()
ret["lora_peft_-1"] = (tox_mean, tox_std, ppl)
peft_model = adapter_scaling(peft_model, "lora", -1.0)

100%|████████████████████████████████████████████████████████████████████████████| 16/16 [01:10<00:00,  4.43s/it]


0.01391994088572801 0.08319615116613897
haha 287644
1


100%|█████████████████████████████████████████████████████████████████████████▋| 560/562 [00:34<00:00, 16.20it/s]

tensor(17.9687, device='cuda:0')


In [57]:
ret

{'lora_pyreft_0': (0.07239165349199084,
  0.20904985270364193,
  16.445749282836914),
 'lora_pyreft_1': (0.2356256846955489, 0.36062346937609063, 18.34659194946289),
 'lora_pyreft_-1': (0.02782047360364004,
  0.11401897784032253,
  18.34307098388672),
 'reft_all_0': (0.07239165349199084, 0.20904985270364193, 16.445749282836914),
 'reft_all_1': (0.2254427645853525, 0.34547832646255394, 16.67818260192871),
 'reft_all_-1': (0.0213432774300486, 0.09923869876326916, 17.288719177246094),
 'reft_f1_0': (0.07239165349199084, 0.20904985270364193, 16.445749282836914),
 'reft_f1_1': (0.27369099783618367, 0.37188162905204397, 17.05349349975586),
 'reft_f1_-1': (0.019930323498101643, 0.09382162088810096, 16.86375617980957),
 'reft_f1_cattn_0': (0.07239165349199084,
  0.20904985270364193,
  16.445749282836914),
 'reft_f1_cattn_1': (0.22378594287988562,
  0.3495321010104451,
  16.724109649658203),
 'reft_f1_cattn_-1': (0.0489121805875925,
  0.15722763116930472,
  18.11237144470215),
 'lora_peft_1': (

### Final Results

In [58]:
import pandas as pd

# Convert dictionary to DataFrame
df = pd.DataFrame.from_dict(ret, orient='index', columns=['Toxicity-Mean', 'Toxicity-STD', 'Perplexity'])

# Display the DataFrame as a table
print(df)


                  Toxicity-Mean  Toxicity-STD  Perplexity
lora_pyreft_0          0.072392      0.209050   16.445749
lora_pyreft_1          0.235626      0.360623   18.346592
lora_pyreft_-1         0.027820      0.114019   18.343071
reft_all_0             0.072392      0.209050   16.445749
reft_all_1             0.225443      0.345478   16.678183
reft_all_-1            0.021343      0.099239   17.288719
reft_f1_0              0.072392      0.209050   16.445749
reft_f1_1              0.273691      0.371882   17.053493
reft_f1_-1             0.019930      0.093822   16.863756
reft_f1_cattn_0        0.072392      0.209050   16.445749
reft_f1_cattn_1        0.223786      0.349532   16.724110
reft_f1_cattn_-1       0.048912      0.157228   18.112371
lora_peft_1            0.199269      0.329436   18.456427
lora_peft_-1           0.013920      0.083196   17.968708
